Discutiremos uma visão geral dos fundamentos do Processamento de Linguagem Natural, que consiste em combinar técnicas de aprendizado de máquina com texto, utilizando matemática/estatística para obter esse texto em um formato que os algoritmos de aprendizado de máquina possam entender.

    
**Requisitos: Você precisará ter o NLTK instalado, juntamente com o download do corpus para palavras irrelevantes (stopwords)**

## Obtenha os dados

Usaremos um conjunto de dados do [UCI datasets](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection). Este conjunto de dados já está localizado na pasta desta seção.

O arquivo que estamos usando contém uma coleção de mais de 5 mil mensagens telefônicas SMS. Você pode conferir o arquivo **readme** para mais informações.

Vamos em frente e usar rstrip() mais uma compreensão de lista para obter uma lista de todas as linhas de mensagens de texto:

In [ ]:
# Esse trecho do código é obrigatório para quem estiver fazendo tudo pelo colab
# Caso você esteja utilizando o jupyter pode comentar/apagar
import os
from google.colab import drive

drive.mount('/content/drive')
os.chdir("drive/My Drive/Colab Notebooks/IA/20_Natural_Language_Processing")
os.listdir()

Uma coleção de textos também é chamada de "corpus". Vamos imprimir as dez primeiras mensagens e numerá-las usando **enumerate**:

Devido ao espaçamento, podemos dizer que este é um [TSV](http://en.wikipedia.org/wiki/Tab-separated_values) ("tab separated values") arquivo, onde a primeira coluna é um rótulo dizendo se a mensagem fornecida é uma mensagem normal (comumente conhecida como "ham") ou "spam". A segunda coluna é a própria mensagem. (Observe que nossos números não fazem parte do arquivo, eles são apenas da chamada **enumerate**).

Usando esses exemplos rotulados de ham e spam, **treinaremos um modelo de aprendizado de máquina para aprender a discriminar entre ham/spam automaticamente**. Então, com um modelo treinado, poderemos **classificar mensagens arbitrárias sem rótulo** como ham ou spam.

Em vez de analisar o TSV manualmente usando Python, podemos aproveitar os pandas.

Usaremos read_csv com o argumento sep, também podemos especificar os nomes das colunas desejadas passando uma lista de nomes.

## Análise exploratória de dados

Vamos conferir algumas das estatísticas com gráficos e os métodos embutidos no pandas

Vamos usar **groupby** para usar descrever por rótulo (label), dessa forma podemos começar a pensar nas características que separam ham e spam

À medida que continuamos nossa análise, queremos começar a pensar nas características que usaremos. Isso está de acordo com a ideia geral de [feature engineering](https://en.wikipedia.org/wiki/Feature_engineering). Quanto melhor seu conhecimento de domínio sobre os dados, melhor sua capacidade de projetar mais carcterísticas a partir deles.

### Visualização de dados

Parece que o comprimento do texto pode ser uma boa característica para se pensar! 

Vamos tentar explicar por que o eixo x vai até 1000, isso deve significar que há uma mensagem muito longa!

910 caracteres! vamos encontrar esta mensagem:

Parece que temos algum tipo de Romeu mandando mensagens!

Mas vamos nos concentrar novamente na ideia de tentar descobrir se o comprimento da mensagem é uma característica distintiva entre ham e spam:

Por meio de um EDA básico, conseguimos descobrir uma tendência de que as mensagens de spam tendem a ter mais caracteres. (Desculpe Romeu!)

Agora vamos começar a processar os dados para que possamos usá-los com o SciKit Learn!

## Pré-processamento de texto

Nosso principal problema com nossos dados é que estão todos em formato de texto (strings). Os algoritmos de classificação que aprendemos até agora precisarão de algum tipo de vetor de recursos numéricos para realizar a tarefa de classificação. Na verdade, existem muitos métodos para converter um corpus em um formato vetorial. O mais simples é a abordagem de [bag-of-words](http://en.wikipedia.org/wiki/Bag-of-words_model), onde cada palavra única em um texto será representada por um número.

Vamos converter as mensagens brutas (sequência de caracteres) em vetores (sequência de números).

Primeiro, vamos escrever uma função que dividirá uma mensagem em suas palavras individuais e retornará uma lista. Também removeremos palavras muito comuns ('the', 'a', etc.). Para isso, vamos aproveitar a biblioteca **NLTK**. Ela é praticamente a biblioteca padrão em Python para processamento de texto e possui muitos recursos úteis. Usaremos apenas alguns dos básicos aqui.

Vamos criar uma função que irá processar a string na coluna de mensagem, então podemos usar **apply()** do pandas para processar todo o texto no DataFrame.

Primeiro removendo a pontuação. Podemos apenas aproveitar a biblioteca **string** incorporada do Python para obter uma lista rápida de todas as pontuações possíveis:


Agora vamos ver como remover palavras irrelevantes. Podemos importar uma lista de palavras irrelevantes em inglês do NLTK (verifique a documentação para mais idiomas e informações).

Vamos colocar os dois juntos em uma função para aplicá-lo ao nosso DataFrame mais tarde:

Aqui está o DataFrame original novamente:

Agora vamos "tokenizar" essas mensagens. Tokenização é apenas o termo usado para descrever o processo de conversão de strings de texto normais em uma lista de tokens (palavras que realmente queremos).

Vamos ver um exemplo de saída na coluna:

**Observação:**
Podemos receber alguns avisos ou erros para símbolos que não consideramos ou que não estavam em Unicode (como um símbolo de libra esterlina)

### Continuação da normalização

Há muitas maneiras de continuar normalizando este texto. Tal como [Stemming](https://en.wikipedia.org/wiki/Stemming) ou distinguir por [parte do discurso](http://www.nltk.org/book/ch05.html).

O NLTK possui muitas ferramentas integradas e ótima documentação sobre muitos desses métodos. Às vezes, eles não funcionam bem para mensagens de texto devido à maneira como muitas pessoas tendem a usar abreviações ou taquigrafias, por exemplo:
    
    'Nah dawg, IDK! Wut time u headin to da club?'
    
versus

    'No dog, I don't know! What time are you heading to the club?'
    
Alguns métodos de normalização de texto terão problemas com esse tipo de abreviação e, portanto, deixarei você explorar esses métodos mais avançados através do [NLTK book online](http://www.nltk.org/book/).

Por enquanto vamos nos concentrar apenas em usar o que temos para converter nossa lista de palavras em um vetor real que o SciKit-Learn pode usar.

## Vetorização

Agora, temos as mensagens como listas de tokens (também conhecidas como [lemmas](http://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html)) e precisamos converter cada uma dessas mensagens em um vetor com o qual os modelos de algoritmo do SciKit Learn possam trabalhar.

Faremos isso em três etapas usando o modelo bag-of-words:

1. Contando quantas vezes uma palavra ocorre em cada mensagem (conhecida como frequência de termo)

2. Pesar as contagens, para que os tokens frequentes tenham um peso menor (frequência inversa do documento)

3. Normalizar os vetores para comprimento unitário, para abstrair do comprimento do texto original (L2 norm)

Vamos começar o primeiro passo:

Cada vetor terá tantas dimensões quantas forem as palavras únicas no corpus SMS. Primeiro, usaremos o **CountVectorizer** do SciKit Learn. Este modelo converterá uma coleção de documentos de texto em uma matriz de contagem de tokens.

Podemos imaginar isso como uma matriz bidimensional. Onde a dimensão 1 é todo o vocabulário (1 linha por palavra) e a outra dimensão são os documentos reais, neste caso uma coluna por mensagem de texto.

Por exemplo:

<table border = “1“>
<tr>
<th></th> <th>Message 1</th> <th>Message 2</th> <th>...</th> <th>Message N</th> 
</tr>
<tr>
<td><b>Word 1 Count</b></td><td>0</td><td>1</td><td>...</td><td>0</td>
</tr>
<tr>
<td><b>Word 2 Count</b></td><td>0</td><td>0</td><td>...</td><td>0</td>
</tr>
<tr>
<td><b>...</b></td> <td>1</td><td>2</td><td>...</td><td>0</td>
</tr>
<tr>
<td><b>Word N Count</b></td> <td>0</td><td>1</td><td>...</td><td>1</td>
</tr>
</table>


Como há tantas mensagens, podemos esperar muitas contagens com zero para a presença de uma dada palavra em um determinado documento. Por causa disso, o SciKit Learn produzirá uma [Sparse Matrix](https://en.wikipedia.org/wiki/Sparse_matrix).

Existem muitos argumentos e parâmetros que podem ser passados ​​para o CountVectorizer. Neste caso vamos apenas especificar que o **analyzer** seja a nossa função previamente definida:

Vamos pegar uma mensagem de texto e obter sua contagem de palavras como um vetor, usando nosso novo `bow_transformer`:

Agora vamos ver sua representação vetorial:

Isso significa que há sete palavras únicas na mensagem número 4 (depois de remover as palavras de parada comuns). Dois deles aparecem duas vezes, o resto apenas uma vez. Vamos verificar e confirmar quais aparecem duas vezes:

Agora podemos usar **.transform** em nosso objeto transformado Bag-of-Words (bow) e transformar todo o DataFrame de mensagens:

Após a contagem, a ponderação e a normalização dos termos podem ser feitas com [TF-IDF](http://en.wikipedia.org/wiki/Tf%E2%80%93idf), utilizando scikit-learn `TfidfTransformer`.

### O que é TF-IDF?
TF-IDF significa *term frequency-inverse document frequency*, e o tf-idf weight é um peso frequentemente usado na recuperação de informações e mineração de texto. Esse peso é uma medida estatística usada para avaliar a importância de uma palavra para um documento em uma coleção ou corpus. A importância aumenta proporcionalmente ao número de vezes que uma palavra aparece no documento, mas é compensada pela frequência da palavra no corpus. Variações do esquema de ponderação tf-idf são frequentemente usadas pelos mecanismos de pesquisa como uma ferramenta central na pontuação e classificação da relevância de um documento dada a consulta do usuário.

Uma das funções de classificação mais simples é calculada somando o tf-idf para cada termo de consulta; muitas funções de classificação mais sofisticadas são variantes desse modelo simples.

Normalmente, o tf-idf weight é composto por dois termos: o primeiro calcula a normalized Term Frequency (TF), aka. o número de vezes que uma palavra aparece em um documento, dividido pelo número total de palavras naquele documento; o segundo termo é a Frequência Inversa de Documentos (IDF), calculada como o logaritmo do número de documentos no corpus dividido pelo número de documentos onde o termo específico aparece.

**TF: Term Frequency**, mede a frequência com que um termo ocorre em um documento. Como cada documento é diferente em tamanho, é possível que um termo apareça muito mais vezes em documentos longos do que em documentos mais curtos. Assim, a frequência do termo é frequentemente dividida pelo comprimento do documento (também conhecido como o número total de termos no documento) como forma de normalização:

*TF(t) = (Número de vezes que o termo t aparece em um documento) / (Número total de termos no documento).*

**IDF: Inverse Document Frequency**, mede a importância de um termo. Ao calcular TF, todos os termos são considerados igualmente importantes. No entanto, sabe-se que certos termos, como “é”, “de” e “isso”, podem aparecer muitas vezes, mas têm pouca importância. Assim, precisamos pesar os termos frequentes enquanto escalamos os raros, calculando o seguinte:

*IDF(t) = log(Número total de documentos / Número de documentos com termo t nele).*

Veja abaixo um exemplo simples.

**Exemplo:**

Considere um documento contendo 100 palavras em que a palavra gato aparece 3 vezes.

O termo frequência (ou seja, tf) para gato é então (3/100) = 0,03. Agora, suponha que temos 10 milhões de documentos e a palavra gato aparece em mil deles. Então, a frequência inversa do documento (ou seja, idf) é calculada como log(10.000.000 / 1.000) = 4. Assim, o peso Tf-idf é o produto dessas quantidades: 0,03 * 4 = 0,12.

Vamos em frente e verificar qual é o IDF (inverse document frequency) 
da palavra `"u"` e de palavra `"university"`?

Para transformar todo o corpus bag-of-words em corpus TF-IDF de uma só vez:

Há muitas maneiras de pré-processar e vetorizar os dados. Essas etapas envolvem feature engineering e construção de um "pipeline". Convido você a verificar a documentação do SciKit Learn sobre como lidar com dados de texto, bem como a ampla coleção de artigos e livros disponíveis sobre o tópico geral de NLP.

## Treinando um modelo

Com as mensagens representadas como vetores, podemos finalmente treinar nosso classificador de spam/ham. Agora podemos usar praticamente qualquer tipo de algoritmo de classificação. Por [uma variedade de razões](http://www.inf.ed.ac.uk/teaching/courses/inf2b/learnnotes/inf2b-learn-note07-2up.pdf), o algoritmo classificador Naive Bayes é uma boa escolha.


Estaremos usando o scikit-learn aqui, escolhendo o [Naive Bayes](http://en.wikipedia.org/wiki/Naive_Bayes_classifier) classificador para começar:

Vamos tentar classificar nossa única mensagem aleatória e verificar como fazemos:

Fantástico! Desenvolvemos um modelo que pode tentar prever a classificação de spam versus ham!

## Avaliação do modelo

Agora queremos determinar o desempenho geral do nosso modelo em todo o conjunto de dados. Vamos começar obtendo todas as previsões:

Podemos usar o relatório de classificação integrado do SciKit Learn, que retorna [precision, recall,](https://en.wikipedia.org/wiki/Precision_and_recall) [f1-score](https://en.wikipedia.org/wiki/F1_score), e uma coluna para suporte (ou seja, quantos casos deram suporte a essa classificação). Confira os links para informações mais detalhadas sobre cada uma dessas métricas e a figura abaixo:

Existem algumas métricas possíveis para avaliar o desempenho do modelo. Qual é o mais importante depende da tarefa e dos efeitos de negócios das decisões baseadas no modelo. Por exemplo, o custo de prever erroneamente "spam" como "ham" é provavelmente muito menor do que prever erroneamente "ham" como "spam".

Na "avaliação" acima, avaliamos a precisão nos mesmos dados que usamos para o treinamento. **Você nunca deve avaliar no mesmo conjunto de dados em que treina!**

Tal avaliação não nos diz nada sobre o verdadeiro poder preditivo do nosso modelo. Se simplesmente lembrássemos de cada exemplo durante o treinamento, a precisão nos dados de treinamento seria trivialmente 100%, mesmo que não pudéssemos classificar nenhuma mensagem nova.

Uma maneira adequada é dividir os dados em um conjunto de treinamento/teste, em que o modelo só vê os **dados de treinamento** durante o ajuste do modelo e o ajuste dos parâmetros. Os **dados de teste** nunca são usados ​​de forma alguma. Esta é, então, nossa avaliação final sobre os dados de teste é representativa do verdadeiro desempenho preditivo.

## Divisão de teste de treino

O tamanho do teste é 20% de todo o conjunto de dados (1.115 mensagens de um total de 5.572), e o treinamento é o restante (4.457 de 5.572). Observe que a divisão padrão teria sido 30/70.

## Criando um pipeline de dados

Vamos executar nosso modelo novamente e, em seguida, prever fora do conjunto de teste. Usaremos o SciKit Learn [pipeline](http://scikit-learn.org/stable/modules/pipeline.html) para armazenar um pipeline de fluxo de trabalho. Isso nos permitirá configurar todas as transformações que faremos nos dados para uso futuro. Vamos ver um exemplo de como funciona:

Agora podemos passar diretamente os dados de texto da mensagem e o pipeline fará nosso pré-processamento para nós! Podemos tratá-lo como uma API de modelo/estimador:

Agora temos um relatório de classificação para nosso modelo em um verdadeiro conjunto de testes!

## Mais recursos

Confira os links abaixo para obter mais informações sobre o Processamento de Linguagem Natural:

[NLTK Book Online](http://www.nltk.org/book/)

[Kaggle Walkthrough](https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words)

[SciKit Learn's Tutorial](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)